<h4>(1 - Recommend). Create and save new notebook(s) to enable automatic re-mounting of Drive storage. Copy and paste the below code into a <a href="http://colab.research.google.com/#create=true" target="_parent">new notebook</a> in Colab and follow the directions in the <a href="../#how-to-use" target="_parent">How To Use</a> section of the <em>README</em></h4><h4>(2). Open the pre-formed version in Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/tdulcet/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title <h1><strong>📔 Colab GPU and CPU Notebook</strong></h1>#{ output-height: 8, form-width: "96%", display-mode: "form" }

#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>🔌 Make sure the GPU is enabled under <em>Runtime→Change runtime type</em></h4>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in the browser or 📌 the tab to a <a href="https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open" target="_blank">dedicated window</a> to prevent disconnection.</h4>
#@markdown <h4>🐛 The <em>debug</em> field outputs GPU (CUDALucas) or CPU (Prime95) progress, then exits.</h4>
#@markdown <h4>ℹ This notebook uses both our CUDALucas and Prime95 <a href="https://github.com/tdulcet/Distributed-Computing-Scripts#organizations" target="_blank">Bash scripts</a> and our <a href="https://github.com/tdulcet/Distributed-Computing-Scripts#primenet" target="_blank">PrimeNet Python script</a>.</h4>
#@markdown <h4>📜 Please see the <a href="https://github.com/tdulcet/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">documentation</a> for more information and to support us.</h4>
#@markdown <h4>🤷 Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here</a> and <a href="https://www.mersenne.org/jteam/" target="_blank">join</a> the “Portland State University” team!</h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
GPU_type_of_work = '100 - First time LL tests' #@param ['100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests']
CPU_type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '153 - 100 million digit PRP tests', '160 - First time PRP on Mersenne cofactors', '161 - Double-check PRP on Mersenne cofactors']
computer_number = "Default (1)" #@param ["Default (1)", "2"] {allow-input: true}
output_type = 'GPU (CUDALucas)' #@param ['GPU (CUDALucas)', 'CPU (Prime95)'] 
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'GPU (CUDALucas)', 'CPU (Prime95)']
gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
path_dir = "" # helps us to not %cd in the optimize_gpu function

import os

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def optimize_gpu():
  '''If a new GPU is being used, optimize CUDALucas for using this GPU'''
  print('\nOptimizing CUDALucas for this computer and GPU\n')
  !cd cudalucas && chmod 777 CUDALucas
  if not os.path.exists(path_dir + 'cudalucas/' + gpu_info + ' fft.txt'):
    !cd cudalucas && ./CUDALucas -cufftbench 1024 8192 5
  if not os.path.exists(path_dir + 'cudalucas/' + gpu_info + ' threads.txt'):
    !cd cudalucas && ./CUDALucas -threadbench 1024 8192 5 0

def run():
  '''Run CUDALucas and MPrime'''
  print('\nStarting PrimeNet\n')
  !cd cudalucas; nohup python3 primenet.py -d -t 10800 -T $GPU_type_of_work -l "{'local' + computer_number + '.ini'}" &
  !sleep 1
  optimize_gpu()
  while not os.path.exists('cudalucas/worktodo' + computer_number+ '.txt'):
    print(f'Waiting for worktodo{computer_number}.txt access...')
    !sleep 1

  if output_type == 'GPU (CUDALucas)':
    print('\nStarting Prime95\n')
    !cd mprime_gpu && chmod 777 mprime; nohup ./mprime -A$computer_number -d >> "{'cpu' + computer_number + '.out'}" &
    !cd cudalucas && ./CUDALucas -i "{'CUDALucas' + computer_number + '.ini'}" | tee -a "{'gpu' + computer_number + '.out'}"
  else:
    print('\nStarting CUDALucas\n')
    !cd cudalucas; nohup ./CUDALucas -i "{'CUDALucas' + computer_number + '.ini'}" >> "{'gpu' + computer_number + '.out'}" &
    !cd mprime_gpu && chmod 777 mprime && ./mprime -A$computer_number -d | tee -a "{'cpu' + computer_number + '.out'}"

def install():
  '''Download/Install/Configure CUDALucas then Prime95'''
  !wget https://github.com/tdulcet/Distributed-Computing-Scripts/archive/master.zip -nv -O master.zip
  !unzip -o master.zip
  
  !cp Distributed-Computing-Scripts-master/{cudalucas2.sh,primenet.py,idletime.sh} .
  !sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas2.sh # Do not check for an Nvidia GPU
  !sed -i '/^[[:blank:]]*if ! COMPUTE=/,/^[[:blank:]]*fi/!b; /^[[:blank:]]*fi/a echo "$COMPUTE"' cudalucas2.sh # Output CUDA compute capability of GPU
  !sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_35,code=sm_35 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' cudalucas2.sh
  !sed -i '/^\.\/CUDALucas / s/^/# /' cudalucas2.sh # Disable optimization step for faster install
  !sed -i '/^nohup / s/^/# /' cudalucas2.sh # Do not start PrimeNet
  !sed -i '/^python3 / s/^/# /' cudalucas2.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/# /' cudalucas2.sh # Do not create a cronjob
  !bash -- cudalucas2.sh $computer_number $prime_ID $computer_name $GPU_type_of_work
  !cd cudalucas && python3 primenet.py -d -t 0 -L 2 -T $GPU_type_of_work -u $prime_ID -i "{'worktodo' + computer_number + '.txt'}" -r "{'results' + computer_number + '.txt'}" -l "{'local' + computer_number + '.ini'}" -g "{'gpu' + computer_number + '.out'}" -H $computer_name
  !cp -u Distributed-Computing-Scripts-master/google-colab/gpu_optimizations/* cudalucas/

  !cp Distributed-Computing-Scripts-master/{mprime2.sh,mprime2.exp} .
  !sed -i 's/"mprime"/"mprime_gpu"/' mprime2.sh # Name the folder specific to the runtime type
  !sed -i '/^nohup / s/^/# /' mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime2.sh # Do not create a cronjob
  !sed -i '/^expect {/a \\t"Upload bandwidth limit in Mbps (*):" { sleep 1; send -- "1000\\r"; exp_continue }\n\t"Skip advanced resource settings (*):" { sleep 1; send -- "n\\r"; exp_continue }\n\t"Optional directory to hold *:" { sleep 1; send -- "\\r"; exp_continue }\n\t"stage 2 memory in GB (*):" { sleep 1; send -- "6\\r"; exp_continue }\n\t"Max emergency memory in GB/worker (*):" { sleep 1; send -- "3\\r"; exp_continue }' mprime2.exp
  !bash -- mprime2.sh $computer_number $prime_ID $computer_name $CPU_type_of_work # Run script
  !{ echo 'FixedHardwareUID=1'; cat "{'mprime_gpu/prim000' + computer_number + '.txt'}"; } > temp.txt
  !mv temp.txt "{'mprime_gpu/prim000' + computer_number + '.txt'}"
  run()

def debug_exit():
  '''Output GPU and output of Prime95 or CUDALucas output'''
  if debug == 'GPU (CUDALucas)'and os.path.exists('cudalucas/gpu' + computer_number + '.out'):
    print(f"\nOutput for computer number {computer_number}:\n")
    print("\nPrimenet output:\n") 
    !tail -n 100 "cudalucas/nohup.out" # view primenet output
    print("\nGPU (CUDALucas) output: ")
    !tail -n 100 "{'cudalucas/gpu' + computer_number + '.out'}" # view CUDALucas progress
    print()
  elif debug == 'CPU (Prime95)' and os.path.exists('mprime_gpu/cpu' + computer_number + '.out'):
    print("\nCPU (Prime95) output:\n") 
    !tail -n 100 "{'mprime_gpu/cpu' + computer_number + '.out'}" # view MPrime progress
    !cd mprime_gpu && chmod 777 mprime && echo -e "3\n\n5" | ./mprime -m
    print()
  else:
    print(f'No `{debug}` output file found for debug option and computer number `{computer_number}`.\n')


def load_drive():
  '''Load & cd into gdrive for persistent data'''
  global path_dir
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd "/content/gdrive/My Drive"
  path_dir = "/content/gdrive/My Drive/GIMPS/"
  if "My Drive/GIMPS" in os.getcwd(): # don't create a subfolder in GIMPS/
    return
  !mkdir -p GIMPS
  %cd "GIMPS"

def gpu_check():
  '''GPU Check'''
  global gpu_info
  gpu_info = "\n".join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Select the "Runtime" → "Change runtime type" → "GPU" → "SAVE" to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
    raise StopExecution
  print(f'\nGraphics Processor (GPU):\t{gpu_info}\n')

gpu_check()
load_drive()

# set local time
!rm -f /etc/localtime
!ln -s {'/usr/share/zoneinfo/US/' + local_time} /etc/localtime

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number
CPU_type_of_work = CPU_type_of_work.split("-")[0].rstrip()
GPU_type_of_work = GPU_type_of_work.split("-")[0].rstrip()
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = f'"'+ prime_ID + f'"'
computer_name = f'"'+ computer_name + f'"'

!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info

if debug:
  debug_exit()
  raise StopExecution

elif not computer_number.isdigit() or int(computer_number) < 0 or int(computer_number) > 9:
  print("ERROR: Computer number must be a number from 0-9")
  raise StopExecution

elif os.path.exists('mprime_gpu/work000' + computer_number + '.txt') and os.path.exists('cudalucas/local' + computer_number + '.ini'):
  run()

else:
  install()

print("Gracefully exiting...")